In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math



df = pd.read_csv('Rotman MMA Summer Datathon.csv')
#filtering for data for women only
women_data = df[df['team_name'].str.contains("Women") & df['team_name'].str.contains("Canada")]
# print (women_data.columns)
#women_data.columns

def div_safe(x,y):
    try:
        return x/y
    except ZeroDivisionError:
        return 0
    
def cal_Distance(x1,y1,x2,y2):
    return round(math.sqrt((x2-x1)**2+(y2-y1)**2),2)


Index(['game_date', 'game_name', 'team_name', 'opp_team_name', 'team_venue',
       'player_name', 'period', 'situation_type', 'event_type',
       'event_successful', 'event_time', 'x_event', 'y_event', 'receiver_name',
       'receiver_x', 'receiver_y', 'shot_type'],
      dtype='object')


In [19]:
#------- Power Sepecialist----------


# {name : [count of t, total count, receive count of t ,  total recieve, [distance list] , [location list[x,y]]}
power_PlayCounts = dict()
# {name : [play success rate, receive success rate ,  average distance, count of edge pass}
power_PlayStats = dict()
#{name : [play success rate, receive success rate ,  average distance}
power_Score = dict()


#Extract data from the sources and orgnaize it in dictionary
for i , j in women_data.iterrows():
    # field of interest
    name = women_data.loc[i]['player_name']
    receiver = women_data.loc[i]['receiver_name']
    
    situation = women_data.loc[i]['situation_type']
    event = women_data.loc[i]['event_type']
    success = women_data.loc[i]['event_successful']
    
    eventX = women_data.loc[i]['x_event']
    eventY = women_data.loc[i]['y_event']
    receiveX = women_data.loc[i]['receiver_x']
    receiveY = women_data.loc[i]['receiver_y']
    if event == 'Play':
        if situation == '5 on 4' or situation == '5 on 3' or situation == '4 on 3':
            if name not in power_PlayCounts:
                power_PlayCounts[name] = [0,0,0,0,[],[]]
            if receiver not in power_PlayCounts: 
                power_PlayCounts[receiver] = [0,0,0,0,[],[]]

            power_PlayCounts[name][1] = power_PlayCounts[name][1] + 1
            power_PlayCounts[receiver][3] = power_PlayCounts[receiver][3] + 1
            if success == 't':
                power_PlayCounts[name][0] = power_PlayCounts[name][0] + 1
                power_PlayCounts[receiver][2] = power_PlayCounts[receiver][2] + 1
                power_PlayCounts[name][4].append(cal_Distance(eventX,eventY,receiveX,receiveY))
                power_PlayCounts[name][5].append([eventX,eventY])
           
# calculate Status
for name,value in power_PlayCounts.items():
    #play success rate
    power_PlayStats[name] = []
    power_PlayStats[name].append(div_safe(value[0],value[1]))
    #recieve success rate
    power_PlayStats[name].append(div_safe(value[2],value[3]))
    #avg distance of success pass
    power_PlayStats[name].append(div_safe(sum(value[4]),len(value[4])))


# print ("All Power: ", power_PlayCounts)
# print ("------------------------")
# print ("Stats: ", power_PlayStats)



for i in range (3):
    j = 0
    temp_value = 0
    temp_score = 100
    for name, value in sorted(power_PlayStats.items(), key=lambda e: e[1][i],reverse = True):
        if name not in power_Score:
            power_Score[name] = [0,0,0]
        if temp_value != value[i]:
            power_Score[name][i] = max (100 - j*5, 0)
            temp_score = power_Score[name][i]
            j += 1
        else:
            power_Score[name][i] = temp_score
        temp_value = value[i]    
#         print (i, name, value, power_Score[name][i])
#     print ("------------")


print ("===========================")
print (power_Score)

0 Ann-Renee Desbiens [1.0, 0, 9.033999999999999] 100
0 Lauriane Rougeau [1.0, 1.0, 67.124] 100
0 Laura Stacey [1.0, 0.8333333333333334, 28.810000000000002] 100
0 Shannon Szabados [1.0, 0, 27.659545454545455] 100
0 Genevieve Lacasse [1.0, 0, 12.276] 100
0 Jamie Lee Rattray [1.0, 1.0, 44.69] 100
0 Rebecca Leslie [1.0, 0.5, 28.13] 100
0 Jocelyne Larocque [0.9411764705882353, 0.875, 38.181875000000005] 95
0 Ann-Sophie Bettez [0.9230769230769231, 0.6666666666666666, 32.36083333333333] 90
0 Marie-Philip Poulin [0.8947368421052632, 0.9629629629629629, 39.85254901960785] 85
0 Melodie Daoust [0.8909090909090909, 0.7352941176470589, 30.598367346938783] 80
0 Loren Gabel [0.8888888888888888, 0.8333333333333334, 32.565000000000005] 75
0 Renata Fast [0.88, 0.95, 34.635909090909095] 70
0 Laura Fortino [0.8769230769230769, 0.9193548387096774, 33.65885964912278] 65
0 Brigette Lacquette [0.875, 0.9514563106796117, 39.2492380952381] 60
0 Blayre Turnbull [0.8571428571428571, 0.6666666666666666, 30.7666666

In [16]:
#------- Penalty Sepecialist----------


# {name : [count of t, total count, receive count of t ,  total recieve, [distance list] , [location list[x,y]]}
penalty_PlayCounts = dict()
# {name : [total count, [game,time] , [location list[x,y]]}
penalty_TakeawayCounts = dict()
# {name : [play success rate, receive success rate ,  average distance, count of edge pass}
penalty_PlayStats = dict()
# {name : [# total count, # at def side]}
penalty_TakeawayStats = dict()
# {name : [play success rate, receive success rate ,average distance,# total count, # at def side]}
penalty_Score = dict()
#{name:score}
player_score = dict()

#Extract data from the sources and orgnaize it in dictionary
for i , j in women_data.iterrows():
    # field of interest
    name = women_data.loc[i]['player_name']
    receiver = women_data.loc[i]['receiver_name']
    
    situation = women_data.loc[i]['situation_type']
    game = women_data.loc[i]['game_name']
    time = women_data.loc[i]['event_time']
    
    event = women_data.loc[i]['event_type']
    success = women_data.loc[i]['event_successful']
    
    eventX = women_data.loc[i]['x_event']
    eventY = women_data.loc[i]['y_event']
    receiveX = women_data.loc[i]['receiver_x']
    receiveY = women_data.loc[i]['receiver_y']
    
    
    if situation == '4 on 5' or situation == '3 on 5' or situation == '3 on 4':
        if event == 'Play':               
            if name not in penalty_PlayCounts:
                penalty_PlayCounts[name] = [0,0,0,0,[],[]]
            if receiver not in penalty_PlayCounts: 
                penalty_PlayCounts[receiver] = [0,0,0,0,[],[]]

            penalty_PlayCounts[name][1] = penalty_PlayCounts[name][1] + 1
            penalty_PlayCounts[receiver][3] = penalty_PlayCounts[receiver][3] + 1
            if success == 't':
                penalty_PlayCounts[name][0] = penalty_PlayCounts[name][0] + 1
                penalty_PlayCounts[receiver][2] = penalty_PlayCounts[receiver][2] + 1
                penalty_PlayCounts[name][4].append(cal_Distance(eventX,eventY,receiveX,receiveY))
                penalty_PlayCounts[name][5].append([eventX,eventY])
                
        elif event == 'Takeaway':
            if name not in penalty_TakeawayCounts:
                penalty_TakeawayCounts[name] = [0,[],[]]
            penalty_TakeawayCounts[name][0] = penalty_TakeawayCounts[name][0] + 1
            penalty_TakeawayCounts[name][1].append([game,time])
            penalty_TakeawayCounts[name][2].append([eventX,eventY])

            
            
# calculate Status
for name,value in penalty_PlayCounts.items():
    #--------------------play ----------------------
    penalty_PlayStats[name] = []
    #play success rate
    penalty_PlayStats[name].append(div_safe(value[0],value[1]))
    #recieve success rate
    penalty_PlayStats[name].append(div_safe(value[2],value[3]))
    #avg distance of success pass
    penalty_PlayStats[name].append(div_safe(sum(value[4]),len(value[4])))
    
for name,value in penalty_TakeawayCounts.items():    
    #--------------------takeaway ----------------------
    penalty_TakeawayStats[name] = []
    #total count 
    penalty_TakeawayStats[name].append(value[0])
    penalty_TakeawayStats[name].append(0)
    for i in penalty_TakeawayCounts[name][2][0]:
        if i < 100:
            penalty_TakeawayStats[name][1] += 1
    
    
# print ("All Play: ", penalty_PlayCounts)
# print ("------------------------")
# print ("Stats: ", penalty_PlayStats)
# print ("===================================")
# print ("All Takeaway: ", penalty_TakeawayCounts)
# print ("------------------------")
# print ("Stats: ", penalty_TakeawayStats)



#assign scores for play
for i in range (3):
    j = 0
    temp_value = 0
    temp_score = 100
    for name, value in sorted(penalty_PlayStats.items(), key=lambda e: e[1][i],reverse = True):
        if name not in penalty_Score:
            penalty_Score[name] = [0,0,0,0,0]
            
        if temp_value != value[i]:
            penalty_Score[name][i] = max (100 - j*5, 0)
            temp_score = penalty_Score[name][i]
            j += 1
        else:
            penalty_Score[name][i] = temp_score
        temp_value = value[i]        
#         print (i, name, value, penalty_Score[name][i])
#     print ("------------")

#assign scores for takeaway
for i in range (2):
    j = 0
    temp_value = 0
    temp_score = 100
    for name, value in sorted(penalty_TakeawayStats.items(), key=lambda e: e[1][i],reverse = True):
        if name not in penalty_Score:
            penalty_Score[name] = [0,0,0,0,0]
            
        if temp_value != value[i]:
            penalty_Score[name][i+3] = max (100 - j*5, 0)
            temp_score = penalty_Score[name][i+3]
            j += 1
        else:
            penalty_Score[name][i+3] = temp_score
        temp_value = value[i]  
#         print (i, name, value, penalty_Score[name][i+3])
#     print ("------------")
    
    
# print ("==================")
#{30%, 15%, 5%, 35%, 15%}     
# {name : [play success rate, receive success rate ,average distance,# total count, # at def side]}
# print ("penalty_Score: ",penalty_Score)


#assigne total score
for name,value in penalty_Score.items():    
    player_score[name] = []
    player_score[name].append(value[0]*0.30+value[1]*0.15+value[2]*0.05+value[3]*0.35+value[3]*0.15)
    
    
print ("==================")
print (sorted(player_score.items(), key=lambda e: e[1],reverse = True))

[('Jocelyne Larocque', [95.25]), ('Erin Ambrose', [91.25]), ('Laura Stacey', [87.75]), ('Halli Krzyzaniak', [87.0]), ('Emily Clark', [83.5]), ('Jillian Saulnier', [82.25]), ('Rebecca Johnston', [82.25]), ('Blayre Turnbull', [81.5]), ('Renata Fast', [80.75]), ('Marie-Philip Poulin', [80.0]), ('Laura Fortino', [78.75]), ('Natalie Spooner', [72.5]), ('Brigette Lacquette', [70.75]), ('Brianne Jenner', [68.75]), ('Victoria Bach', [43.25]), ('Lauriane Rougeau', [41.75]), ('Meaghan Mikkelson', [41.25]), ('Meghan Agosta', [40.5]), ('Shannon Szabados', [35.0]), ('Bailey Bram', [33.25]), ('Ann-Sophie Bettez', [33.25]), ('Sarah Nurse', [29.75]), ('Mich Zandee-Hart', [26.5])]
